# **Rohlik Sales Prediction**

# **Dataset 설명**

 - 선택한 Rohlik 재고 및 날짜에 대한 과거 판매 데이터가 제공됩니다. ID, 판매, 총 주문 및 가격 열은 실제 값을 기밀로 유지하기 위해 변경됩니다. 일부 기능은 예측을 하는 시점에는 알 수 없으므로 테스트에서 사용할 수 없습니다. 작업은 테스트 세트에 대해 unique_id와 date(예: unique_id 1226과 date 2024-06-03에서 id 1226_2024-06-03)로 구성된 주어진 id에 대한 판매 열을 예측하는 것입니다.

파일

 - sales_train.csv - 아래에 설명된 선택된 기능을 갖춘 지정된 날짜 및 재고에 대한 과거 판매 데이터가 포함된 교육 세트
 - sales_test.csv - 전체 테스트 세트
 - inventory.csv - 제품과 같은 재고에 대한 추가 정보(모든 창고의 동일한 제품은 동일한 제품 고유 ID 및 이름을 공유하지만 고유 ID는 다름)
 - solution.csv - 올바른 형식의 전체 제출 파일
 - calendar.csv - 휴일 또는 창고별 이벤트에 대한 데이터가 포함된 달력, 일부 열은 이미 열차 데이터에 있지만 공휴일 또는 일요일로 인해 일부 창고가 닫힐 수 있는 날짜에 대한 추가 행이 이 파일에 있습니다(따라서 열차 세트에 없음)

# **Column 설명**

 # sales_train.csv 및 sales_test.csv
 - unique_id- 재고에 대한 고유 ID
 - date - 날짜
 - warehouse- 창고 이름
 - total_orders- 이 도전의 일부로 테스트 세트로 알려진 선택된 Rohlik 창고에 대한 과거 주문
 - sales- 목표 값, 판매 볼륨(개 또는 kg)은 가용성에 따라 조정됩니다. 1보다 낮은 가용성을 가진 판매는 이미 Rohlik 내부 논리에 의해 전체 잠재 판매로 조정됩니다. 다양한 이유로 주어진 재고에 대한 train과 test에서 날짜가 누락될 수 있습니다. 이 열은 test.csv에서 누락되었는데, 이는 목표 변수이기 때문입니다.
 - sell_price_main- 판매 가격
 - availability- 재고가 고객에게 제공될 수 있었던 하루의 비율. 재고는 항상 제공될 필요는 없습니다. 값 1은 하루 종일 제공되었음을 의미합니다. 이 열은 예측을 하는 시점에는 알 수 없기 때문에 test.csv에 없습니다.
 - type_0_discount, type_1_discount, … - Rohlik은 다양한 유형의 프로모 세일 액션을 실행하고 있으며, 이는 프로모 기간 동안 원래 가격의 백분율을 보여줍니다(( original price- current_price) / original_price). 다양한 유형의 여러 할인을 동시에 실행할 수 있지만, 이러한 유형 중 가장 높은 할인이 항상 세일에 사용됩니다. 음수 할인 값은 할인 없음으로 해석해야 합니다.

 # 인벤토리.csv
 - unique_id - 단일 보관 단위에 대한 인벤토리 ID
 - product_unique_id - 제품 ID, 각 창고의 재고는 동일한 제품 고유 ID를 갖습니다(모든 창고의 동일한 제품은 동일한 제품 ID를 갖지만 고유 ID는 다름)
 - name - 단일 보관 단위에 대한 인벤토리 ID
 - L1_category_name, L2_category_name, … - 내부 카테고리의 이름입니다. 숫자가 높을수록 더 세부적인 정보가 제공됩니다.
 - warehouse - 창고 이름

 # 달력.csv
 - warehouse - 창고 이름
 - date - 날짜
 - holiday_name - 공휴일이 있는 경우 공휴일 이름
 - holiday - 0/1은 휴일이 있음을 나타냅니다.
 - shops_closed - 대부분의 상점 또는 상당수의 상점이 문을 닫는 공휴일
 - winter_school_holidays - 겨울방학
 - school_holidays - 학교 방학

 # 테스트_가중치.csv
 - unique_id - 단일 보관 단위에 대한 인벤토리 ID
 - weight- 최종 메트릭 계산에 사용되는 가중치

# 1. EDA(탐색적 데이터 분석)

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [4]:
# 데이터 로드
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Rohlik Sales Prediction/sales_train.csv')
inventory = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Rohlik Sales Prediction/inventory.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Rohlik Sales Prediction/calendar.csv')

In [5]:
# 날짜 데이터 변환
train['date'] = pd.to_datetime(train['date'])
calendar['date'] = pd.to_datetime(calendar['date'])

In [6]:
# train 데이터와 calendar 데이터 병합 (휴일 정보 추가)
train = pd.merge(train, calendar, on=['date', 'warehouse'], how='left')

In [7]:
# inventory 데이터와 병합 (상품 정보 추가)
train = pd.merge(train, inventory, on='unique_id', how='left')

In [8]:
# 샘플링할 데이터 개수 (조절 가능)
sample_size = 10000

In [9]:
# train 데이터 샘플링
train_sample = train.sample(sample_size, random_state=42)

In [17]:
# 1. 휴일 종류별 매출 변화 분석 (L2 카테고리 사용)

def analyze_sales_by_holiday_l2(df):
    # 특정 기간 데이터 필터링 (예시: 2024년 1분기)
    df_filtered = df[(df['date'] >= '2024-01-01') & (df['date'] <= '2024-03-31')]
    holiday_sales = df_filtered.groupby(['holiday_name', 'L2_category_name_en'])['sales'].sum().reset_index()
    fig = px.bar(holiday_sales, x='holiday_name', y='sales', color='L2_category_name_en', title='Sales by Holiday Type and L2 Product Category (Sample Data)')
    fig.show()

In [26]:
# 2. 할인 정책 종류별 매출 변화 분석 (상품명 사용)

def analyze_sales_by_discount_item_optimized(df, top_n=20):
    discount_columns = ['type_0_discount', 'type_1_discount', 'type_2_discount', 'type_3_discount', 'type_4_discount', 'type_5_discount', 'type_6_discount']
    df['max_discount'] = df[discount_columns].max(axis=1)

    # 1. 상위 판매 상품 분석
    top_items = df.groupby('name')['sales'].sum().nlargest(top_n).index
    df_top = df[df['name'].isin(top_items)]
    discount_sales_top = df_top.groupby(['max_discount', 'name'])['sales'].sum().reset_index()
    fig_top = px.bar(discount_sales_top, x='max_discount', y='sales', color='name', title=f'Sales by Discount Rate and Top {top_n} Items (Sample Data)')
    fig_top.show()

    # 2. 카테고리별 분석
    discount_sales_category = df.groupby(['max_discount', 'L2_category_name_en'])['sales'].sum().reset_index()
    fig_category = px.bar(discount_sales_category, x='max_discount', y='sales', color='L2_category_name_en', title='Sales by Discount Rate and L2 Category (Sample Data)')
    fig_category.show()

    # 3. 히트맵 시각화 (카테고리별)
    discount_sales_category = df.groupby(['max_discount', 'L2_category_name'])['sales'].sum().reset_index()
    discount_sales_category_pivot = discount_sales_category.pivot(index='max_discount', columns='L2_category_name', values='sales').fillna(0)
    fig_heatmap = px.imshow(discount_sales_category_pivot, title='Sales Heatmap by Discount Rate and L2 Category (Sample Data)')
    fig_heatmap.show()

    return discount_sales_category_pivot # pivot table 반환

In [28]:
# 3. 여러 창고 또는 여러 품목의 판매 추이 비교 분석

def compare_sales_trends(df, warehouses=None, items=None):
    if warehouses is None:
        warehouses = df['warehouse'].unique()
    if items is None:
        items = df['name'].unique()

    # 비교할 창고 및 품목 선택 (예시)
    selected_warehouses = ['Praha', 'Brno']
    selected_items = ['Banana', 'Apple']

    # 선택된 창고 또는 특정 품목만 비교
    df_filtered = df[df['warehouse'].isin(selected_warehouses) & df['name'].isin(selected_items)]
    fig = px.line(df_filtered, x='date', y='sales', color='warehouse', facet_col='name', title='Sales Trend Comparison (Sample Data)')
    fig.show()

In [18]:
# 1. 휴일 종류별 매출 변화 분석
analyze_sales_by_holiday_l2(train_sample)

In [32]:
# 2. 할인 정책 종류별 매출 변화 분석 (최적화)
pivot_table = analyze_sales_by_discount_item_optimized(train_sample, top_n=10) # pivot table 저장

In [29]:
# 3. 여러 창고 또는 여러 품목의 판매 추이 비교 분석
compare_sales_trends(train_sample) # 인자 생략 시 기본값으로 실행

KeyError: 'warehouse'

# 2. 데이터 로드 및 전처리

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# 데이터 로드
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Rohlik Sales Prediction/sales_train.csv')
inventory = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Rohlik Sales Prediction/inventory.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Rohlik Sales Prediction/calendar.csv')
test_weights = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Rohlik Sales Prediction/test_weights.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Rohlik Sales Prediction/sales_test.csv')

In [3]:
# 날짜 데이터 변환
train['date'] = pd.to_datetime(train['date'])
calendar['date'] = pd.to_datetime(calendar['date'])
test['date'] = pd.to_datetime(test['date'])

In [4]:
# train 데이터 전처리
train = pd.merge(train, inventory, on='unique_id', how='left', suffixes=('', '_inventory')) # 접미사 지정
train.head(5)

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,type_4_discount,type_5_discount,type_6_discount,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse_inventory
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,...,0.15312,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,...,0.15025,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,...,0.15025,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,...,0.15312,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,...,0.15649,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1


In [5]:
train = pd.merge(train, calendar, on=['date', 'warehouse'], how='left') # calendar 데이터 병합(상품정보 추가)
train.head(5)

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse_inventory,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,NaN,0,0,0,0
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,NaN,0,0,0,0
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,NaN,0,0,0,0
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,NaN,0,0,0,0
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,NaN,0,0,0,0


In [6]:
# 할인율 컬럼 생성 (최대 할인율 사용)
discount_columns = ['type_0_discount', 'type_1_discount', 'type_2_discount', 'type_3_discount', 'type_4_discount', 'type_5_discount', 'type_6_discount']
train['max_discount'] = train[discount_columns].max(axis=1)

# 결측치 처리 (예시: 0으로 채우기)
train.fillna(0, inplace=True)

In [10]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Rohlik Sales Prediction/sales_test.csv')

In [11]:
test['date'] = pd.to_datetime(test['date'])

In [12]:
# test 데이터 전처리 (개선된 부분)
test = pd.merge(test, inventory, on='unique_id', how='left', suffixes=('', '_inventory')) # 접미사 지정
test.head()

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse_inventory
0,1226,2024-06-03,Brno_1,8679.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,Brno_1
1,1226,2024-06-11,Brno_1,8795.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,Brno_1
2,1226,2024-06-13,Brno_1,10009.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,Brno_1
3,1226,2024-06-15,Brno_1,8482.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,Brno_1
4,1226,2024-06-09,Brno_1,8195.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,Brno_1


In [13]:
test = pd.merge(test, calendar, left_on='date', right_on='date', how='left')
test['max_discount'] = test[discount_columns].max(axis=1)
test['holiday'] = test['holiday'].apply(lambda x: 1 if x != 0 else 0)  # holiday 컬럼 생성
test.fillna(0, inplace=True)

 - 공휴일(holiday) 변수 생성 및 분석

In [14]:
# holiday 컬럼을 0과 1로 변환 (공휴일이면 1, 아니면 0)
train['holiday'] = train['holiday'].apply(lambda x: 1 if x != 0 else 0)

In [15]:
# 공휴일별 판매량 차이 분석
holiday_sales = train.groupby('holiday')['sales'].mean()
print(holiday_sales)

holiday
0    109.077355
1     91.965467
Name: sales, dtype: float64


 - 할인율 변수 추가 및 분석

In [16]:
# 할인율별 판매량 분석
discount_sales = train.groupby('max_discount')['sales'].mean()
print(discount_sales)

max_discount
0.00000    102.968923
0.00002     85.520000
0.00003     26.577500
0.00004     45.020000
0.00005     21.206667
              ...    
0.90004     17.402500
0.90005      3.290000
0.95527     40.922857
0.97345    100.380000
0.99164    212.830000
Name: sales, Length: 46359, dtype: float64


 - 간단한 예측 모델 생성 (Linear Regression)

In [17]:
# 모델 학습에 사용할 Feature 선택
features = ['holiday', 'max_discount']  # 추가 변수 포함
target = 'sales'

In [18]:
# Linear Regression 모델 학습
model = LinearRegression()
model.fit(train[features], train[target])

LinearRegression()

In [19]:
# test 데이터 예측
test_predictions = model.predict(test[features])
test_predictions[test_predictions < 0] = 0

In [20]:
# 가중치 적용된 RMSE 계산 함수
def weighted_rmse(y_true, y_pred, weights):
    return np.sqrt(np.average((y_true - y_pred)**2, weights=weights))

# 가중치 적용된 MAE 계산 함수
def weighted_mae(y_true, y_pred, weights):
    return np.average(np.abs(y_true - y_pred), weights=weights)

# train 데이터에 test_weights 병합 (unique_id 기준)
train_weighted = pd.merge(train, test_weights, on='unique_id', how='left')
train_weighted

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,L3_category_name_en,L4_category_name_en,warehouse_inventory,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,max_discount,weight
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,0,0,0,0,0,0.15312,1.925596
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,0,0,0,0,0,0.15025,1.925596
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,0,0,0,0,0,0.15025,1.925596
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,0,0,0,0,0,0.20024,1.925596
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,0,0,0,0,0,0.15649,1.925596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007414,4941,2023-06-21,Prague_1,9988.0,26.56,34.06,1.00,0.00000,0.0,0.0,...,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1,Prague_1,0,0,0,0,0,0.00000,2.262646
4007415,4941,2023-06-24,Prague_1,8518.0,27.42,34.06,1.00,0.00000,0.0,0.0,...,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1,Prague_1,0,0,0,0,0,0.00000,2.262646
4007416,4941,2023-06-23,Prague_1,10424.0,33.39,34.06,1.00,0.00000,0.0,0.0,...,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1,Prague_1,0,0,0,0,0,0.00000,2.262646
4007417,4941,2023-06-22,Prague_1,10342.0,22.88,34.06,1.00,0.00000,0.0,0.0,...,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1,Prague_1,0,0,0,0,0,0.00000,2.262646


In [21]:
# 가중치 적용된 RMSE, MAE 계산
w_rmse = weighted_rmse(train_weighted['sales'], train['sales'], train_weighted['weight'])
w_mae = weighted_mae(train_weighted['sales'], train['sales'], train_weighted['weight'])

print(f'Weighted RMSE: {w_rmse}')
print(f'Weighted MAE: {w_mae}')

Weighted RMSE: 0.0
Weighted MAE: 0.0


In [70]:
# 제출 파일 생성 (길이가 같은지 확인 후 생성)
if len(test_predictions) == len(test):
    submission = pd.DataFrame({'id': test['id'].values, 'sales_hat': test_predictions}) # .values 추가
    submission.to_csv('submission.csv', index=False)
    print('submission.csv 파일 생성 완료')
else:
    print("Error: test_predictions and test data have different lengths.")
    print(f"Length of test_predictions: {len(test_predictions)}")
    print(f"Length of test_final: {len(test)}")

KeyError: 'id'